# ⚽ Expected vs. Reality: Is xG a True Predictor of Goals & Can Teams Use it to Gain Tactical Insight?
---


## 🧹 2) Data Cleansing 
---

### 📍 1. Import Libraries


In [123]:
import pandas as pd

### 📍 2. Load the Data


In [126]:
# Load the raw Premier League xG dataset
df = pd.read_csv(r'C:\Users\d_par\OneDrive\Desktop\Danny\2025\Data Science\Portolio Projects\prem-xg-analysis\Data\premier-league-24.csv')

#Print out a preview of the data to display the format
print("Raw Data Preview:")
display(df.head())


Raw Data Preview:


,Day,Date,Time,Home,xG,Score,xG.1,Away
0,Fri,2023-08-11,20:00,Burnley,0.3,0–3,1.9,Manchester City
1,Sat,2023-08-12,12:30,Arsenal,0.8,2–1,1.2,Nott'ham Forest
2,Sat,2023-08-12,15:00,Everton,2.7,0–1,1.5,Fulham
3,Sat,2023-08-12,15:00,Sheffield Utd,0.5,0–1,1.9,Crystal Palace
4,Sat,2023-08-12,15:00,Brighton,4.0,4–1,1.5,Luton Town


### 📍 3. Clean Column Names

In [129]:
# Rename columns for clarity
df.rename(columns={'xG': 'Home_xg', 'xG.1': 'Away_xg'}, inplace=True)

### 📍 4. Create 23/24 Rankings and Merge to Main Dataframe

In [143]:

# Final league positions for each team
rankings_data = {
    'Ranking': range(1, 21),  # Assigns positions from 1 to 20
    'Team': [
        'Manchester City', 'Arsenal', 'Liverpool', 'Aston Villa', 'Tottenham', 'Chelsea', 
        'Newcastle Utd', 'Manchester Utd', 'West Ham', 'Crystal Palace', 'Brighton', 'Bournemouth', 'Fulham' , 'Wolves', 'Everton', 'Brentford',
          'Nott\'ham Forest', 'Luton Town', 'Burnley',  'Sheffield Utd'
    ]
}

rankings_df = pd.DataFrame(rankings_data)

# Save rankings for use in other scripts
rankings_df.to_csv('Rankings.csv', index=False)


# Merge rankings for Home team in fixtures
df = df.merge(rankings_df, left_on='Home', right_on='Team', how='outer').rename(columns={'Ranking': 'Home Rank'})
df.drop(columns=['Team'], inplace=True)

# Merge rankings for Away team in fixtures
df = df.merge(rankings_df, left_on='Away', right_on='Team', how='outer').rename(columns={'Ranking': 'Away Rank'})
df.drop(columns=['Team'], inplace=True)


#Add Points onto the Rankings for use further down the line
rankings_df['Points'] = [91,89,82,68,66,63,60,60,52,49,48,48,47,46,40,39,32,26,24,16]

# Save rankings for use in other scripts
rankings_df.to_csv(r'C:\Users\d_par\OneDrive\Desktop\Danny\2025\Data Science\Portolio Projects\prem-xg-analysis\Data\Rankings.csv', index=False)


### 📍 5. Handle Missing Values

##### It became apparent the df had read in a number of redundant, empty rows. These are removed here.

In [146]:
# Identify and remove empty rows
rows_with_nulls = df[df.isnull().any(axis=1)]

# Display rows with missing data (commented out to not take up space)
#display(rows_with_nulls) 

df.dropna(inplace=True)


### 📍 6. Process Score Column & Add New Collumns for Goals Scored and xG Error

###### Here, xG Error is defined as xG - Goals. Meaning a positive 'Error' value indicates xG overestimates the number of goals scored, where as a negative value indicates xG underestimate the number of goals scored. 

In [149]:

# Split 'Score' column into Home and Away goals
df[['Home_Goals', 'Away_Goals']] = df['Score'].str.split('–', expand=True)

# Convert to integer dtype
df['Home_Goals'] = df['Home_Goals'].astype(int)
df['Away_Goals'] = df['Away_Goals'].astype(int)

# Calculate xG Errors. 
# Difference between expected goals (xG) and actual goals
df['Home xg error'] = df['Home_xg'] - df['Home_Goals']
df['Away xg error'] = df['Away_xg'] - df['Away_Goals']


#additional columns for total goals and xg error
df['Total Goals'] = df['Home_Goals'] + df['Away_Goals']
df['Total xG'] = df['Home_xg'] + df['Away_xg']
df['xG - goals'] = (df['Total xG'] - df['Total Goals'])


### 📍 7.  Save Cleaned Data

In [152]:
df.to_csv(r'C:\Users\d_par\OneDrive\Desktop\Danny\2025\Data Science\Portolio Projects\prem-xg-analysis\Data\cleaned_prem_data.csv', index=False)

print("Data cleaning complete. Cleaned data saved as 'cleaned_prem_data.csv'.")


Data cleaning complete. Cleaned data saved as 'cleaned_prem_data.csv'.
